<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 

Follow instructions step by step until the end and submit your complete notebook as an archive (tar -cf groupXnotebook.tar DL_lab2/).

Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by May 29th 2019 (23:59:59 CET).

__BERTELLI FLORIAN__

__REBOULLET PAUL__

# Introduction

In the last Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%.  Can  you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to  more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
from tensorflow.contrib.layers import flatten
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:

#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288298870
Epoch:  02   =====> Loss= 0.732601715
Epoch:  03   =====> Loss= 0.600517497
Epoch:  04   =====> Loss= 0.536845536
Epoch:  05   =====> Loss= 0.497668130
Epoch:  06   =====> Loss= 0.471057313
Epoch:  07   =====> Loss= 0.451252412
Epoch:  08   =====> Loss= 0.436074483
Epoch:  09   =====> Loss= 0.423697097
Epoch:  10   =====> Loss= 0.413106918
Epoch:  11   =====> Loss= 0.404530114
Epoch:  12   =====> Loss= 0.396962555
Epoch:  13   =====> Loss= 0.390427339
Epoch:  14   =====> Loss= 0.384620576
Epoch:  15   =====> Loss= 0.379247611
Epoch:  16   =====> Loss= 0.374332983
Epoch:  17   =====> Loss= 0.370459973
Epoch:  18   =====> Loss= 0.366480190
Epoch:  19   =====> Loss= 0.362761056
Epoch:  20   =====> Loss= 0.359741725
Epoch:  21   =====> Loss= 0.356469466
Epoch:  22   =====> Loss= 0.354022222
Epoch:  23   =====> Loss= 0.351207959
Epoch:  24   =====> Loss= 0.348940905
Epoch:  25   =====> Loss= 0.346488114
Epoch:  26   =====> Loss= 0.344253443
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png" width="800" height="600" align="center">





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [2]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [3]:
def LeNet5_Model(image): 
    
    #Defining the firdt convolutional layer
    conv1 = tf.nn.conv2d(
        input=image,
        filter=weight_variable((5, 5, 1, 6)), 
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv1'
    )
    
    #Adding the activation function and the pooling layer
    conv1 = tf.nn.relu(conv1 + bias_variable([1, 28, 28, 6]))
    max_pool1 = tf.nn.max_pool(
        value=conv1,
        ksize=[1,2,2,1],
        strides=[1,2,2,1], #H2=(H1-K)/S +1 (output shape 1x14x14x6)
        padding='VALID',
        name='max_pool1'
    )
    
    #Defining the second convolutional layer
    conv2 = tf.nn.conv2d(
        input=max_pool1,
        filter=weight_variable((5, 5, 6, 16)), 
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv2'
    )
    
    #Adding the activation function and the pooling layer
    conv2 = tf.nn.relu(conv2 + bias_variable([1, 10, 10, 16]))
    max_pool2 = tf.nn.max_pool(
        value=conv2,
        ksize=[1,2,2,1],
        strides=[1,2,2,1], #H2=(H1-K)/S +1 (output shape 1x5x5x16)
        padding='VALID',
        name='max_pool2'
    )
    
    #Flatten the layer
    flat = flatten(max_pool2)
 
    
    # Fully Connected with 120 neurons with ReLu
    dense1 = tf.layers.dense(
        inputs=flat,
        units=120,
        activation=tf.nn.relu,
        use_bias=True,
        bias_initializer=tf.constant_initializer(value=1.0),
        name='dense11'
    )

    
    # Fully Connected with 84 neurons with ReLu
    dense2 = tf.layers.dense(
        inputs=dense1,
        units=84,
        activation=tf.nn.relu,
        use_bias=True,
        bias_initializer=tf.constant_initializer(value=1.0),
        name='dense21'
    )

    
    # Fully Connected with 10 neurons with SoftMax
    dense3 = tf.layers.dense(
        inputs=dense2,
        units=10,
        activation=tf.nn.softmax,
        use_bias=True,
        bias_initializer=tf.constant_initializer(value=1.0),
        name='dense31'
    )
    
    # Return predictions
    return dense3

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

In our implementation we have :

    - conv 1  + bias :  5*5*1 + 6
    - conv 2  + bias : 5*5*6 + 16
    - dense 1 + bias : 5*5*16*120 + 120
    - dense 2 + bias : 84*120 + 84
    - dense 3 + bias : 10*84 +10
    
  
Indeed, for each layer we have to sum the nummber of weight and the bias we had.
    
    
The total of parameters is 59331

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [5]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'log_files_/'



<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [6]:
def evaluate(model, y):
    correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    return tf.reduce_mean(tf.cast(correct, tf.float32))

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [7]:
import time
def train(learning_rate, training_epochs, batch_size, display_step = 1, \
          logs_path='log_files_/', optimizerF="SGD", verbose=True , transfer='sigmoid'):
    start = time.time()
    optimizer_dict = {"SGD":tf.train.GradientDescentOptimizer, "Adam":tf.train.AdamOptimizer}
    
    # Erase previous graph
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')


    # Construct model
    with tf.name_scope('Model'):
        pred = LeNet5_Model(x)

    # Define loss and optimizer
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

    with tf.name_scope(optimizerF):
        if transfer is "sigmoid":
            optimizer = optimizer_dict[optimizerF](learning_rate).minimize(cost)
        else:
            opt = optimizer_dict[optimizerF](learning_rate)
            gvs = opt.compute_gradients(cost)
            capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
            optimizer = opt.apply_gradients(capped_gvs)

    # Evaluate model
    with tf.name_scope('Accuracy'):
        accuracy = evaluate(pred, y)

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", accuracy)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    x_val, y_val = mnist.validation.images.reshape(-1, 28, 28, 1), mnist.validation.labels
    x_test, y_test = mnist.test.images.reshape(-1, 28, 28, 1), mnist.test.labels

    with tf.Session() as sess:
#         acc_history = []
        test_history = []
        val_history = []
#         train_history = []

        sess.run(init)
        if verbose is True:
            print("Start Training!")
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        saver = tf.train.Saver()
        #Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            #Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                batch_xs = batch_xs.reshape(-1, 28, 28, 1)
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys})

                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
#             train_acc = accuracy.eval({x: batch_xs, y:batch_ys})
            val_acc = accuracy.eval({x: x_val, y:y_val})
            test_acc = accuracy.eval({x: x_test, y:y_test})
#             acc_history.append(acc)
#             train_history.append(train_acc)
            val_history.append(val_acc)
            test_history.append(test_acc)
            
            saver.save(sess, 'Models/model_' + str(learning_rate) + '_' + str(batch_size) + '_' + optimizerF)
            if verbose is True and (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), \
                      "  =====> Loss=", "{:.9f}".format(avg_cost), \
                      " Validation accuracy=", val_acc, " Test accuracy=", test_acc)
            if val_acc>=0.99:
                if verbose is True:
                    print("Validation Accuracy over 99%% reached after %d epochs" %(epoch+1))
                break
        print('It takes ' , time.time()-start , 'to train')  
        start = time.time()
        if verbose is True:
            print("Training Finished!")
            # Test model
            # Calculate accuracy
            print("Test accuracy:", accuracy.eval({x: x_test, y:y_test}))
        print('It takes ' , time.time()-start , 'to test')  
    return val_history, test_history

In [8]:
# trainining with first configuration
val_hist, test_hist = train(learning_rate, training_epochs, batch_size)

Start Training!
Epoch:  01   =====> Loss= 2.359788304  Validation accuracy= 0.3548  Test accuracy= 0.345
Epoch:  02   =====> Loss= 1.984217592  Validation accuracy= 0.5362  Test accuracy= 0.5385
Epoch:  03   =====> Loss= 1.494585602  Validation accuracy= 0.7168  Test accuracy= 0.7205
Epoch:  04   =====> Loss= 0.966587138  Validation accuracy= 0.8036  Test accuracy= 0.806
Epoch:  05   =====> Loss= 0.663597779  Validation accuracy= 0.8454  Test accuracy= 0.8508
Epoch:  06   =====> Loss= 0.523834231  Validation accuracy= 0.87  Test accuracy= 0.8733
Epoch:  07   =====> Loss= 0.449842068  Validation accuracy= 0.8832  Test accuracy= 0.887
Epoch:  08   =====> Loss= 0.403340158  Validation accuracy= 0.892  Test accuracy= 0.8951
Epoch:  09   =====> Loss= 0.370185190  Validation accuracy= 0.9006  Test accuracy= 0.8994
Epoch:  10   =====> Loss= 0.346036175  Validation accuracy= 0.9048  Test accuracy= 0.9077
Epoch:  11   =====> Loss= 0.325403106  Validation accuracy= 0.9092  Test accuracy= 0.9127


<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

<img src="./MNIST_figures/TF_Data_WithOut_Dropout.png" width="600" height="400" align="center">


__COMMENTS:__

As we can see the accuracy is growing as we follow the gradient in the decreasing way. In opposite, our loss is decreasing 

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |      0.958       |      0.992          |       
| Training Time (second)       |         703        |        162          |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** 

The optimizer which gives the best accuracy on the Test Data is the AdamOptimizer (0.992 vs 0.9655) for a shorter training time too.
Indeed, compared to the Gradient Descent Optimizer, Adam Optimizer instead of adapting the parameter learning rates based on the average first moment (the mean) as in RMSProp, Adam also makes use of the average of the second moments of the gradients (the uncentered variance).

Specifically, the algorithm calculates an exponential moving average of the gradient and the squared gradient, and the parameters beta1 and beta2 control the decay rates of these moving averages.

This leads to have a moving learning rate constructed to reach the global minimum. Gradient Descent's learning rate doesn't move at all and so it is not very tuned for complicated minimun.


In [14]:
# trainining with first configuration

tf.reset_default_graph()
# your implementation goes here

val_hist, test_hist = train(learning_rate, training_epochs, batch_size,optimizerF='Adam' )

Start Training!
Epoch:  01   =====> Loss= 0.341464647  Validation accuracy= 0.9686  Test accuracy= 0.9702
Epoch:  02   =====> Loss= 0.073869712  Validation accuracy= 0.9808  Test accuracy= 0.9788
Epoch:  03   =====> Loss= 0.052109897  Validation accuracy= 0.983  Test accuracy= 0.983
Epoch:  04   =====> Loss= 0.039654047  Validation accuracy= 0.9876  Test accuracy= 0.9869
Epoch:  05   =====> Loss= 0.033007712  Validation accuracy= 0.9884  Test accuracy= 0.9886
Epoch:  06   =====> Loss= 0.026240505  Validation accuracy= 0.9892  Test accuracy= 0.9894
Epoch:  07   =====> Loss= 0.020087567  Validation accuracy= 0.9892  Test accuracy= 0.988
Epoch:  08   =====> Loss= 0.018770608  Validation accuracy= 0.989  Test accuracy= 0.987
Epoch:  09   =====> Loss= 0.016544443  Validation accuracy= 0.9882  Test accuracy= 0.9884
Epoch:  10   =====> Loss= 0.012987009  Validation accuracy= 0.9908  Test accuracy= 0.9905
Validation Accuracy over 99% reached after 10 epochs
It takes  156.5672001838684 to train

<img src="./MNIST_figures/TF_Data_Adam.png" width="600" height="400" align="center">



__COMMENTS:__

As we can see the curves with adam optimizer increase faster (for the accuracy) and the loss decreases faster too compared to the one with a fixed learning rate.

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** 

- Whitout Adam :

It takes the same time to train (about 626 seconds) but the accuracy is less good than without dropout (0.946 vs 0.9611)

- With Adam:

It takes more time to train (about 340 vs 80 seconds) and the accuracy is the same as the end.




In [10]:
def LeNet5_Model_Dropout(image , probability):    
    conv1 = tf.nn.conv2d(
        input=image,
        filter=weight_variable((5, 5, 1, 6)), 
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv1'
    )
    
    conv1 = tf.nn.relu(conv1 + bias_variable([1, 28, 28, 6]))


    max_pool1 = tf.nn.max_pool(
        value=conv1,
        ksize=[1,2,2,1],
        strides=[1,2,2,1], #H2=(H1-K)/S +1 (output shape 1x14x14x6)
        padding='VALID',
        name='max_pool1'
    )
    
    
    conv2 = tf.nn.conv2d(
        input=max_pool1,
        filter=weight_variable((5, 5, 6, 16)), 
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv2'
    )

    
    conv2 = tf.nn.relu(conv2 + bias_variable([1, 10, 10, 16]))
 
        
    max_pool2 = tf.nn.max_pool(
        value=conv2,
        ksize=[1,2,2,1],
        strides=[1,2,2,1], #H2=(H1-K)/S +1 (output shape 1x5x5x16)
        padding='VALID',
        name='max_pool2'
    )
   
    
    flat = flatten(max_pool2)
 
    #Adding dropout before the first dense layer
    dropout_layer = tf.nn.dropout(
    flat,
    keep_prob=probability)
    
    # Fully Connected with 120 neurons
    dense1 = tf.layers.dense(
        inputs=dropout_layer,
        units=120,
        activation=tf.nn.relu,
        use_bias=True,
        bias_initializer=tf.constant_initializer(value=1.0),
        name='dense11'
    )

    
    # Fully Connected with 84 neurons
    dense2 = tf.layers.dense(
        inputs=dense1,
        units=84,
        activation=tf.nn.relu,
        use_bias=True,
        bias_initializer=tf.constant_initializer(value=1.0),
        name='dense21'
    )

    
    # Fully Connected with 10 neurons
    dense3 = tf.layers.dense(
        inputs=dense2,
        units=10,
        activation=tf.nn.softmax,
        use_bias=True,
        bias_initializer=tf.constant_initializer(value=1.0),
        name='dense31'
    )
    
    # Return predictions
    return dense3

tf.reset_default_graph()
# your implementation goes here

In [11]:
import time
def train_DropOut(probability, learning_rate, training_epochs, batch_size, display_step = 1, \
          logs_path='log_files_DropOut/', optimizerF="SGD", verbose=True , transfer='sigmoid'):
    start = time.time()
    optimizer_dict = {"SGD":tf.train.GradientDescentOptimizer, "Adam":tf.train.AdamOptimizer}
    
    # Erase previous graph
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')


    # Construct model
    with tf.name_scope('Model'):
        pred = LeNet5_Model_Dropout(x , probability)

    # Define loss and optimizer
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

    with tf.name_scope(optimizerF):
        if transfer is "sigmoid":
            optimizer = optimizer_dict[optimizerF](learning_rate).minimize(cost)
        else:
            opt = optimizer_dict[optimizerF](learning_rate)
            gvs = opt.compute_gradients(cost)
            capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
            optimizer = opt.apply_gradients(capped_gvs)

    # Evaluate model
    with tf.name_scope('Accuracy'):
        accuracy = evaluate(pred, y)

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", accuracy)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    x_val, y_val = mnist.validation.images.reshape(-1, 28, 28, 1), mnist.validation.labels
    x_test, y_test = mnist.test.images.reshape(-1, 28, 28, 1), mnist.test.labels

    with tf.Session() as sess:
#         acc_history = []
        test_history = []
        val_history = []
#         train_history = []

        sess.run(init)
        if verbose is True:
            print("Start Training!")
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        saver = tf.train.Saver()
        #Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            #Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                batch_xs = batch_xs.reshape(-1, 28, 28, 1)
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys})

                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
#             train_acc = accuracy.eval({x: batch_xs, y:batch_ys})
            val_acc = accuracy.eval({x: x_val, y:y_val})
            test_acc = accuracy.eval({x: x_test, y:y_test})
#             acc_history.append(acc)
#             train_history.append(train_acc)
            val_history.append(val_acc)
            test_history.append(test_acc)
            
            saver.save(sess, 'Models/model_' + str(learning_rate) + '_' + str(batch_size) + '_' + optimizerF)
            if verbose is True and (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), \
                      "  =====> Loss=", "{:.9f}".format(avg_cost), \
                      " Validation accuracy=", val_acc, " Test accuracy=", test_acc)
            if val_acc>=0.99:
                if verbose is True:
                    print("Validation Accuracy over 99%% reached after %d epochs" %(epoch+1))
                break
        print('It takes ' , time.time()-start , 'to train')  
        start = time.time()
        if verbose is True:
            print("Training Finished!")
            # Test model
            # Calculate accuracy
            print("Test accuracy:", accuracy.eval({x: x_test, y:y_test}))
        print('It takes ' , time.time()-start , 'to test')  
    return val_history, test_history

In [12]:
tf.reset_default_graph()
val_hist, test_hist = train_DropOut(0.75 , learning_rate, training_epochs, batch_size )

Start Training!
Epoch:  01   =====> Loss= 2.355519645  Validation accuracy= 0.2622  Test accuracy= 0.2526
Epoch:  02   =====> Loss= 2.065052887  Validation accuracy= 0.4762  Test accuracy= 0.4774
Epoch:  03   =====> Loss= 1.630875981  Validation accuracy= 0.6156  Test accuracy= 0.6257
Epoch:  04   =====> Loss= 1.179595354  Validation accuracy= 0.7122  Test accuracy= 0.7129
Epoch:  05   =====> Loss= 0.896893010  Validation accuracy= 0.7656  Test accuracy= 0.7733
Epoch:  06   =====> Loss= 0.738787021  Validation accuracy= 0.8038  Test accuracy= 0.8117
Epoch:  07   =====> Loss= 0.634889965  Validation accuracy= 0.8212  Test accuracy= 0.832
Epoch:  08   =====> Loss= 0.562248279  Validation accuracy= 0.8466  Test accuracy= 0.8501
Epoch:  09   =====> Loss= 0.513525600  Validation accuracy= 0.8588  Test accuracy= 0.8606
Epoch:  10   =====> Loss= 0.473145507  Validation accuracy= 0.869  Test accuracy= 0.881
Epoch:  11   =====> Loss= 0.435199199  Validation accuracy= 0.8752  Test accuracy= 0.88

In [13]:
tf.reset_default_graph()
val_hist, test_hist = train_DropOut(0.75 , learning_rate, training_epochs, batch_size ,  optimizerF="Adam" )

Start Training!
Epoch:  01   =====> Loss= 0.405675828  Validation accuracy= 0.9642  Test accuracy= 0.9632
Epoch:  02   =====> Loss= 0.109867787  Validation accuracy= 0.9748  Test accuracy= 0.9706
Epoch:  03   =====> Loss= 0.080308498  Validation accuracy= 0.978  Test accuracy= 0.9772
Epoch:  04   =====> Loss= 0.062533649  Validation accuracy= 0.9816  Test accuracy= 0.9804
Epoch:  05   =====> Loss= 0.057777045  Validation accuracy= 0.9824  Test accuracy= 0.9823
Epoch:  06   =====> Loss= 0.050314618  Validation accuracy= 0.9838  Test accuracy= 0.984
Epoch:  07   =====> Loss= 0.044219773  Validation accuracy= 0.9848  Test accuracy= 0.984
Epoch:  08   =====> Loss= 0.041478408  Validation accuracy= 0.9834  Test accuracy= 0.9839
Epoch:  09   =====> Loss= 0.033542314  Validation accuracy= 0.9842  Test accuracy= 0.9846
Epoch:  10   =====> Loss= 0.032803432  Validation accuracy= 0.988  Test accuracy= 0.9888
Epoch:  11   =====> Loss= 0.030590624  Validation accuracy= 0.9864  Test accuracy= 0.987

<img src="./MNIST_figures/TF_Data_With_Droupout.png" width="600" height="400" align="center">



As we can see the on the two curves, the fastest one is the one with Adam and the other one without